In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
!pip install transformers

You should consider upgrading via the '/opt/anaconda3/envs/dlearn/bin/python -m pip install --upgrade pip' command.


In [24]:
import pandas as pd

from iqradre.detect.pred import BoxesPredictor

from iqradre.recog.prod import TextPredictor
from iqradre.recog.prod import utils as text_utils

import transformers
from transformers import BertTokenizer
from iqradre.extract.prod.prod import Extractor

import matplotlib.pyplot as plt
from iqradre.detect.ops import boxes as boxes_ops
from iqradre.detect.ops import box_ops

ImportError: cannot import name 'LayoutLMConfig'

In [28]:
 from transformers import LayoutLMConfig

ImportError: cannot import name 'LayoutLMConfig'

In [19]:
import numpy as np
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rotate

from deskew import determine_skew
import cv2 as cv

In [ ]:
impath = 'images/test1.png'
img = cv.imread(impath)
shape = img.shape[:2]
shape.index(max(shape)), shape

In [ ]:
craft_weight = f'../weights/detect/craft_ktp_ohem.pth.tar'
crnn_path = '../weights/recog/ocrnet_pretrained_ktp.pth'
layoutlm_path = '../weights/extract/layoutlm_v2.pth'

In [ ]:
predictor = BoxesPredictor(weight_path=craft_weight, device='cpu')
text_pred = TextPredictor(weight_path=crnn_path)
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
extract = Extractor(tokenizer=tokenizer, weight=layoutlm_path)

In [ ]:
%%time
impath = 'images/ktp_aan/R_ktp6.jpg'
# impath = transform.do_autocrop_deskew(impath)
result = predictor.predict_word_boxes(impath, text_threshold=0.3, low_text=0.2)
polys, boxes, images_patch, img, score_text, score_link, ret_score_text = result

In [ ]:
plt.imshow(score_text);plt.show()
plt.imshow(score_link);plt.show()
plt.imshow(score_text+score_link);plt.show()

In [4]:
# image = io.imread('input.png')
# grayscale = rgb2gray(image)
angle = determine_skew(score_text+score_link)
rotated = rotate(img, angle, resize=True)
# io.imsave('output.png', rotated.astype(np.uint8))

plt.imshow(rotated)
img=rotated
print(angle)

NameError: name 'determine_skew' is not defined

In [5]:
boxes_list = box_ops.batch_box_coordinate_to_xyminmax(boxes, to_int=True).tolist()   
boxes_list[0]

NameError: name 'box_ops' is not defined

In [6]:
# import numpy as np
# boxes_proc = np.array(boxes_list)
# xmin, ymin = np.min(boxes_proc[:, 0]), np.min(boxes_proc[:, 1])
# xmax, ymax = np.max(boxes_proc[:, 2]), np.max(boxes_proc[:, 3])

# xmin, ymin, xmax, ymax

In [7]:
# !pip install deskew

In [8]:
idx = 0
plt.imshow(img);plt.show()
plt.imshow(images_patch[idx]);plt.show()
print(boxes_list[idx])

NameError: name 'plt' is not defined

In [9]:
%%time
text_list = text_pred.predict(images_patch)

NameError: name 'text_pred' is not defined

In [10]:
for text, img in zip(text_list, images_patch):
    print(text)    
    plt.imshow(img);plt.show()


NameError: name 'text_list' is not defined

In [11]:
data_annoset = text_utils.build_annoset(text_list, boxes)
data_annoset = sorted(data_annoset, key = lambda i: (i['bbox'][1], i['bbox'][0]))

NameError: name 'text_list' is not defined

In [54]:
data_annoset

[{'text': 'PROVINSI', 'bbox': [438.0, 203.0, 585.0, 234.0]},
 {'text': 'DKI', 'bbox': [588.0, 208.0, 642.0, 234.0]},
 {'text': 'JAKARTA', 'bbox': [644.0, 208.0, 780.0, 236.0]},
 {'text': 'JAKARTA', 'bbox': [488.0, 232.0, 626.0, 258.0]},
 {'text': 'TIMUR', 'bbox': [630.0, 234.0, 728.0, 260.0]},
 {'text': 'NIK', 'bbox': [176.0, 260.0, 242.0, 292.0]},
 {'text': '3175052405810008', 'bbox': [354.0, 263.0, 733.0, 302.0]},
 {'text': 'I.', 'bbox': [316.0, 270.0, 332.0, 290.0]},
 {'text': 'Nama', 'bbox': [166.0, 306.0, 230.0, 328.0]},
 {'text': 'ANDREAS', 'bbox': [442.0, 311.0, 551.0, 335.0]},
 {'text': 'ATWIS', 'bbox': [366.0, 312.0, 440.0, 332.0]},
 {'text': 'HADI', 'bbox': [554.0, 314.0, 612.0, 336.0]},
 {'text': 'IW', 'bbox': [356.0, 316.0, 366.0, 332.0]},
 {'text': 'S--', 'bbox': [612.0, 316.0, 630.0, 336.0]},
 {'text': 'Tempat/Tgl', 'bbox': [164.0, 329.0, 283.0, 356.0]},
 {'text': 'Lahir', 'bbox': [284.0, 334.0, 340.0, 354.0]},
 {'text': 'NGAWI,', 'bbox': [364.0, 334.0, 449.0, 358.0]},
 {

In [55]:
data, clean = extract.predict(data_annoset)

IndexError: The :obj:`bbox`coordinate values should be within 0-1000 range.

In [51]:
data

NameError: name 'data' is not defined

In [35]:
pd.set_option("max_rows", None)
pd.DataFrame(clean)

NameError: name 'clean' is not defined

In [ ]:
trans